In [8]:
# Adopted from 
# https://github.com/watson-developer-cloud/python-sdk/blob/master/examples/visual_recognition_v4.py

import json
import os
from pprint import pprint
from ibm_watson import VisualRecognitionV4
from ibm_watson.visual_recognition_v4 import FileWithMetadata, TrainingDataObject, Location, AnalyzeEnums
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator


In [3]:
authenticator = IAMAuthenticator('UNj_AQojzpzeg5q5FvmaMu2a3jathuqDB79_DjpAGJb_')

service = VisualRecognitionV4(
    '2020-03-21',
    authenticator=authenticator)
service.set_service_url('https://api.us-south.visual-recognition.watson.cloud.ibm.com/instances/de0d73f8-faaa-4318-9ed6-492a2822bf20')

# create a classifier
my_collection = service.create_collection(
    name='face_detection_test',
    description='testing for face detection training'
).get_result()

collection_id = my_collection.get('collection_id')


In [45]:
print(collection_id)

80a7fa45-ab8e-44f5-8455-d5fa1f64053c


### Prepare files for training

In [24]:
annotation_path = "/Users/dqin/Documents/FAME/watson_experiment/resources/FDDB-folds"
image_path = "/Users/dqin/Documents/FAME/watson_experiment/resources/originalPics"

all_data = dict()

anno_files = sorted([x for x in os.listdir(annotation_path) if 'transformed' in x])


for f in anno_files:
    with open(os.path.join(annotation_path, f), "r") as f:
        cur_img_name = ""
        for line in f:
            if "img_" in line:
                cur_img_name = line.strip()
                all_data[cur_img_name] = dict()
                all_data[cur_img_name]['faces'] = []
                
            elif line.strip().isdigit():
                all_data[cur_img_name]['count'] = int(line.strip())
                continue
            else:
                location = [int(float(x)) for x in line.strip().split(", ")]
                all_data[cur_img_name]['faces'].append(location)


In [27]:
# Randomly Select a subset of 10 images for training...
# Intentially select images that have multple faces.
import random
random.seed(1)

single_img_ids = [k for k in all_data if all_data[k]['count']==1]
multi_img_ids = [k for k in all_data if all_data[k]['count']>1]
# len(multi_img_ids):1145 len(all_data):2845

img_names = random.sample(multi_img_ids, 10)
# img_names = random.sample(all_data.keys(), 10)
# all_data[img_names[0]]


In [28]:
img_names

['2002/08/01/big/img_2146',
 '2002/08/09/big/img_224',
 '2002/07/26/big/img_669',
 '2002/08/04/big/img_474',
 '2003/01/17/big/img_884',
 '2002/08/18/big/img_484',
 '2002/08/05/big/img_3752',
 '2002/08/11/big/img_121',
 '2002/07/20/big/img_708',
 '2002/08/27/big/img_19919']

### Upload image to IBM

In [36]:
# for img in img_names:
#     img_path = os.path.join(image_path, img)
img_paths = [os.path.join(image_path, n + ".jpg") for n in img_names]

def return_f_obj(file_name):
    with open(file_name, "rb") as f:
        return f.read()  # directly return the read bytes

add_images_result = service.add_images(
    collection_id,
    images_file=[FileWithMetadata(return_f_obj(f)) for f in img_paths],
).get_result()

print(json.dumps(add_images_result, indent=2))
# image_id = add_images_result.get('images')[0].get('image_id')

{
  "images": [
    {
      "image_id": "image_06ec2c6be57ba98cf6d90b45e3654984",
      "created": "2020-03-22T12:09:17.494Z",
      "updated": "2020-03-22T12:09:17.494Z",
      "source": {
        "type": "file"
      },
      "dimensions": {
        "height": 450,
        "width": 294
      },
      "training_data": {}
    },
    {
      "image_id": "image_59a5a9502602c1cad5a2a9f62c57f7a3",
      "created": "2020-03-22T12:09:13.942Z",
      "updated": "2020-03-22T12:09:13.942Z",
      "source": {
        "type": "file"
      },
      "dimensions": {
        "height": 348,
        "width": 410
      },
      "training_data": {}
    },
    {
      "image_id": "image_7b8c87ebc5e88f80cb154ecc85c0c90c",
      "created": "2020-03-22T12:09:16.978Z",
      "updated": "2020-03-22T12:09:16.978Z",
      "source": {
        "type": "file"
      },
      "dimensions": {
        "height": 409,
        "width": 450
      },
      "training_data": {}
    },
    {
      "image_id": "image_7f74fc33b41

In [41]:
image_ids = [img['image_id'] for img in add_images_result.get('images')]
image_ids

['image_06ec2c6be57ba98cf6d90b45e3654984',
 'image_59a5a9502602c1cad5a2a9f62c57f7a3',
 'image_7b8c87ebc5e88f80cb154ecc85c0c90c',
 'image_7f74fc33b410f8f6695a719869e4749a',
 'image_8d7d681bab19521aa9756dbd5c123f43',
 'image_d9789d9fa7b92551659f08bad7f6afc0',
 'image_d9fd774c3048762abfb17d2646085b0c',
 'image_ecbe194c9f2131b46513ea7618f63dfe',
 'image_f3fa6fe5289cc845a01582cc00a09875',
 'image_f4353a01b214b9e00ab402b4e6b9d1b0']

In [ ]:
# Manual check on IBM service, confirmed? No, cannot confirm on GUI...

### Add in labels?

In [ ]:
# add image training data (label out the )

for img, image_id in zip(img_names, image_ids):
    training_data = service.add_image_training_data(
        collection_id,
        image_id,
        objects=[
            TrainingDataObject(object='giraffe training data',
                               location=Location(64, 270, 755, 784))
        ]).get_result()
    print(json.dumps(training_data, indent=2))


In [44]:
for img in img_names:
    print(img)
    print(all_data[img])

2002/08/01/big/img_2146
{'faces': [[21, 9, 38, 63], [8, 48, 25, 38], [2, 141, 28, 40], [58, 116, 50, 83], [142, 47, 37, 52], [171, 161, 71, 117], [32, 273, 38, 69], [-14, 386, 33, 57]], 'count': 8}
2002/08/09/big/img_224
{'faces': [[30, 192, 74, 98], [136, 140, 33, 49]], 'count': 2}
2002/07/26/big/img_669
{'faces': [[41, 42, 74, 106], [35, 268, 77, 111]], 'count': 2}
2002/08/04/big/img_474
{'faces': [[-2, 74, 97, 137], [10, 296, 88, 129]], 'count': 2}
2003/01/17/big/img_884
{'faces': [[30, -17, 69, 111], [64, 60, 82, 89], [12, 230, 95, 138]], 'count': 3}
2002/08/18/big/img_484
{'faces': [[87, 58, 85, 119], [104, 193, 72, 103], [24, 304, 110, 165], [71, 243, 50, 64], [47, 180, 63, 94], [98, 402, 47, 60], [123, 31, 23, 30]], 'count': 7}
2002/08/05/big/img_3752
{'faces': [[15, 31, 179, 254], [32, 262, 159, 244]], 'count': 2}
2002/08/11/big/img_121
{'faces': [[217, 115, 75, 121], [217, 51, 50, 70]], 'count': 2}
2002/07/20/big/img_708
{'faces': [[38, 73, 129, 181], [31, 252, 117, 172], [69,